# Análise Exploratória dos Dados dos Conglomerados Pruendenciais

O Banco Central do Brasil disponibiliza dados das instituições sob sua supervisão, porém esses dados não são apresentados de maneira clara e objetiva. Embora estejam no formato de valores separados por vírgulas (CSV), eles ainda requerem um tratamento prévio para ser possível extrair informações relevantes de forma eficaz.

Neste notebook, realizaremos uma análise exploratória dos dados referentes aos conglomerados prudenciais e instituições independentes. Um conglomerado prudencial, conforme a Resolução BCB nº 168, de 1º de dezembro de 2021, é definido como um grupo composto por uma instituição de financeira que detém controle, direto ou indireto, sobre outras entidades no Brasil ou no exterior. Essas entidades podem incluir instituições financeiras, outras instituições autorizadas a funcionar pelo Banco Central do Brasil (BCB), instituições de pagamento não autorizadas pelo BCB, entidades que realizam aquisições de operações de crédito ou de direitos creditórios, além de outras pessoas jurídicas que tenham por objetivo exclusivo a participação societária nessas entidades e fundos de investimento. O controle é caracterizado quando a instituição investidora tem poder sobre a entidade controlada, podendo influenciar seus retornos e decisões. A instituição líder do conglomerado é responsável pela consolidação das demonstrações financeiras do grupo, e o Banco Central pode, a seu critério, incluir ou excluir entidades do conglomerado para garantir a correta representação do patrimônio consolidado.

In [1]:
# Importando as bibliotecas necessárias.

# Pandas: manipulação e análise de dados.
import pandas as pd

In [2]:
# Configurando o Pandas para exibir mais dados.
# Exibir todas as colunas (padrão é 20).
pd.set_option('display.max_columns', None)

# Definir largura máxima da coluna (padrão é 50).
pd.set_option('display.max_colwidth', 60)
# Definir precisão de exibição para números flutuantes.
pd.set_option('display.float_format', '{:.2f}'.format)

# Reduzir a quantidade de dados exibidos em DataFrames muito grandes.
pd.set_option('display.large_repr', 'truncate')

In [3]:
# Definindo o caminho do arquivo a ser carregado.
DATA_PATH = '../if_data_processed'
INSTITUTION_TYPE = 'prudential_conglomerates'
REPORT_TYPE = 'summary'
DATA_BASE = '2024-06.csv'
FILE_PATH = f'{DATA_PATH}/{INSTITUTION_TYPE}/{REPORT_TYPE}/{DATA_BASE}'

In [4]:
# Definindo os nomes das colunas do DataFrame, conforme a descrição das colunas.
COLUMN_NAMES = [
    'Instituicao',  # Nome da instituição ou conglomerado no cadastro do Banco Central.
    'Codigo',  # Código da instituição ou conglomerado no cadastro do Banco Central.
    'TipoConsBancario',  # Tipo de Consolidado Bancário.
    'Segmento',  # Segmento conforme Resolução n.º 4.553/2017.
    'TipoConsolidacao',  # Tipo de Consolidação (I) Instituição Independente, (C) Conglomerado.
    'TipoControle',  # Tipo de Controle (1) Público, (2) Privado Nacional, (3) Controle Estrangeiro.
    'Cidade',  # # Cidade da sede da instituição.
    'UF',  # Unidade da Federação onde fica a sede da instituição.
    'DataBase',  # Data-base do relatório.
    'AtivoTotal',  # Ativo Circulante e Realizável a Longo Prazo + Ativo Permanente.
    'CarteiraCredito',  # Carteira de Crédito Classificada.
    'PassivoExigivel',  # Passivo Circulante e Exigível a Longo Prazo e Resultado de Exercícios Futuros.
    'Captacoes',  # Captações de depósitos + Obrigações por Operações Compromissadas 
                  # + Recursos de Aceites Cambiais, Letras Imobiliárias e Hipotecárias,
                  # Debêntures e Similares + Obrigações por Empréstimos e Repasses.
    'PatrimonioLiquido',  # Patrimônio Líquido + Contas de Resultado Credoras
                          # + Contas de Resultado Devedoras. 
    'LucroLiquido',  # Lucro Líquido, excluindo despesas de juros sobre capital.
    'PatrimonioReferencia',  # Montante de capital regulatório formado pela soma
                             # das parcelas de Capital Nível I e Capital Nível II.
    'IndiceBasileia',  # Relação entre o Patrimônio de Referência e Ativos ponderados pelo risco.
    'IndiceImobilizacao',  # Relação entre Ativo Permanente e Patrimônio de Referência.
    'NumAgencias',  # Número de agências incluídas as sedes
    'NumPostosAtendimento'  # Número de postos de atendimento da instituição ou conglomerado
]

In [5]:
# Definindo as descrições dos valores das colunas tipo de consolidado bancário.
descricoes_tipo_consolidado = {
    "b1": "Banco Comercial ou Conglomerado com Carteira Comercial",
    "b2": "Banco Múltiplo sem Carteira Comercial ou Banco de Câmbio/Investimento",
    "b3S": "Cooperativa de Crédito Singular",
    "b3C": "Central e Confederação de Cooperativas de Crédito",
    "b4": "Banco de Desenvolvimento",
    "n1": "Instituição Não Bancária no Mercado de Crédito",
    "n2": "Instituição Não Bancária no Mercado de Capitais",
    "n4": "Instituição de Pagamento"
}

# Definindo as descrições dos valores das colunas tipo de consolidação.
descricoes_tipo_consolidacao = {
    "I": "Instituição Independente",
    "C": "Conglomerado"
}

# Definindo as descrições dos valores das colunas tipo de controle.
descricoes_tipo_controle = {
    1: "Público",
    2: "Privado Nacional",
    3: "Controle Estrangeiro"
}

# Definindo as regiões e estados do Brasil.
regioes_estados = {
    'Norte': ['AC', 'AM', 'AP', 'PA', 'RO', 'RR', 'TO'],  # Ordem alfabética
    'Nordeste': ['AL', 'BA', 'CE', 'MA', 'PB', 'PE', 'PI', 'RN', 'SE'],  # Ordem alfabética
    'Sul': ['PR', 'RS', 'SC'],  # Ordem alfabética
    'Sudeste': ['ES', 'MG', 'RJ', 'SP'],  # Ordem alfabética
    'Centro-oeste': ['DF', 'GO', 'MT', 'MS']  # Ordem alfabética
}
# Mapeando os estados para as regiões do Brasil.
# Formato: {estado: regiao}
estado_para_regiao = {estado: regiao for regiao, estados in regioes_estados.items() for estado in estados}

In [6]:
# Carregando a base de dados para o DataFrame.
# Importante: O arquivo CSV está separado por ponto e vírgula.
df = pd.read_csv(FILE_PATH,
                 sep=';',
                 names=COLUMN_NAMES)

In [7]:
# Primeiro contato com o DataFrame.
df

,Instituicao,Codigo,TipoConsBancario,Segmento,TipoConsolidacao,TipoControle,Cidade,UF,DataBase,AtivoTotal,CarteiraCredito,PassivoExigivel,Captacoes,PatrimonioLiquido,LucroLiquido,PatrimonioReferencia,IndiceBasileia,IndiceImobilizacao,NumAgencias,NumPostosAtendimento
0,ITAU - PRUDENCIAL,1000080099,b1,S1,C,2,SAO PAULO,SP,06/2024,2.636.882.506,946.459.806,2.443.588.602,1.884.371.314,193.293.904,19.963.578,215.556.905,"16,56%","20,86%",2.15,1.10
1,BB - PRUDENCIAL,1000080329,b1,S1,C,1,BRASILIA,DF,06/2024,2.334.367.943,1.024.504.848,2.161.635.722,1.845.389.366,172.732.221,18.173.027,175.348.301,"14,19%","16,35%",4.01,512.00
2,CAIXA ECONÔMICA FEDERAL - PRUDENCIAL,1000080738,b1,S1,C,1,BRASILIA,DF,06/2024,1.907.044.008,1.175.742.235,1.807.735.638,1.617.770.675,99.308.370,5.300.681,129.887.763,"16,40%","9,26%",3.38,846.00
3,BRADESCO - PRUDENCIAL,1000080075,b1,S1,C,2,OSASCO,SP,06/2024,1.682.417.433,658.552.579,1.521.287.541,1.277.833.567,161.129.891,8.937.210,149.119.407,"15,23%","25,57%",2.61,1.05
4,SANTANDER - PRUDENCIAL,1000080185,b1,S1,C,3,SAO PAULO,SP,06/2024,1.279.020.308,540.556.671,1.180.740.535,900.469.651,98.279.773,6.356.898,98.725.721,"14,38%","20,02%",2.53,1.48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1352,COOPERATIVA CENTRAL DE CRÉDITO COM INTERAÇÃO SOLIDÁRIA -...,10013534,b3C,S4,I,2,LAURO DE FREITAS,BA,06/2024,NI,NI,NI,NI,NI,NI,NI,NI%,NI%,0.00,0.00
1353,COOPERATIVA DE CRÉDITO RURAL DO AGRESTE ALAGOANO - COOPE...,8482873,b3S,S5,I,2,IGACI,AL,06/2024,NI,NI,NI,NI,NI,NI,NI,NI%,NI%,0.00,0.00
1354,COOPERATIVA DE CRÉDITO MÚTUO DOS SERVIDORES MUNICIPAIS D...,5969937,b3S,S5,I,2,BEBEDOURO,SP,06/2024,NI,NI,NI,NI,NI,NI,NI,NI%,NI%,0.00,0.00
1355,COOPERATIVA DE ECONOMIA E CRÉDITO MÚTUO DOS BANCÁRIOS DO...,1301541,b3S,S5,I,2,UBERLANDIA,MG,06/2024,NI,NI,NI,NI,NI,NI,NI,NI%,NI%,0.00,0.00


In [8]:
# Verificando o número de colunas e linhas do DataFrame.
print(f'Número de colunas: {df.shape[1]}')
print(f'Número de linhas: {df.shape[0]}')

Número de colunas: 20
Número de linhas: 1357


In [9]:
# Verificando se o Pandas inferiu corretamente os tipos de dados das colunas.
df.dtypes

Instituicao              object
Codigo                    int64
TipoConsBancario         object
Segmento                 object
TipoConsolidacao         object
TipoControle              int64
Cidade                   object
UF                       object
DataBase                 object
AtivoTotal               object
CarteiraCredito          object
PassivoExigivel          object
Captacoes                object
PatrimonioLiquido        object
LucroLiquido             object
PatrimonioReferencia     object
IndiceBasileia           object
IndiceImobilizacao       object
NumAgencias             float64
NumPostosAtendimento    float64
dtype: object

## Transformação e Limpeza dos Dados

Dados reais nem sempre estão prontos para serem analisados. Eles podem conter erros, valores ausentes, formatação inadequada e outras imperfeições que precisam ser tratadas antes de serem utilizadas. Nesta seção, vamos realizar algunas transformações e limpezas nos dados para garantir que eles estejam prontos para a análise exploratória.

In [10]:
# Removendo a parte ' - PRUDENCIAL' da coluna 'Instituicao'.
df['Instituicao'] = df['Instituicao'].str.replace(' - PRUDENCIAL', '')

In [11]:
# A documentação do Banco Central informa que os dados das colunas
# 'TipoConsBancario', 'TipoConsolidacao' e 'TipoControle' são categóricos.
print(f'Tipos de Consolidado Bancário: {df['TipoConsBancario'].unique().tolist()}')
print(f'Segmento Resolução nº 4.553/2017: {df['TipoConsolidacao'].unique().tolist()}')
print(f'Tipo de Consolidação: {df['TipoControle'].unique().tolist()}')

Tipos de Consolidado Bancário: ['b1', 'b4', 'n1', 'n4', 'b2', 'n2', 'b3C', 'b3S']
Segmento Resolução nº 4.553/2017: ['C', 'I']
Tipo de Consolidação: [2, 1, 3]


In [12]:
# A documentação do Banco Central informa que os dados das colunas
# AtivoTotal, CarteiraCredito, PassivoExigivel, Captacoes, PatrimonioLiquido
# e LucroLiquido são numéricos.
print(f'Exemplo de dados da coluna Ativo Total: {df['AtivoTotal'].loc[0]}')
print(f'Exemplo de dados da coluna Carteira de Crédito Classificada: {df['CarteiraCredito'].loc[0]}')
print(f'Exemplo de dados da coluna Passivo Circulante: {df['PassivoExigivel'].loc[0]}')
print(f'Exemplo de dados da coluna Captações: {df['Captacoes'].loc[0]}')
print(f'Exemplo de dados da coluna Patrimônio Líquido: {df['PatrimonioLiquido'].loc[0]}')
print(f'Exemplo de dados da coluna Lucro Líquido: {df['LucroLiquido'].loc[0]}')

Exemplo de dados da coluna Ativo Total: 2.636.882.506
Exemplo de dados da coluna Carteira de Crédito Classificada: 946.459.806
Exemplo de dados da coluna Passivo Circulante: 2.443.588.602
Exemplo de dados da coluna Captações: 1.884.371.314
Exemplo de dados da coluna Patrimônio Líquido: 193.293.904
Exemplo de dados da coluna Lucro Líquido: 19.963.578


In [13]:
# A documentação do Banco Central informa que os dados das colunas
# 'IndiceBasileia' e 'IndiceImobilizacao' são numéricos, representando um percentual.
print(f'Exemplo de dados da coluna Índice de Basileia: {df['IndiceBasileia'].loc[0]}')
print(f'Exemplo de dados da coluna Índice de Imobilização: {df['IndiceImobilizacao'].loc[0]}')

Exemplo de dados da coluna Índice de Basileia: 16,56%
Exemplo de dados da coluna Índice de Imobilização: 20,86%


In [14]:
def clean_number_format(column: pd.Series) -> pd.Series:
    """
    Limpa a formatação de números em uma coluna do DataFrame.
    
    Parameters:
        column (pandas.Series): A coluna a ser limpa.
    """
    
    # Remover o ponto como separador de milhares e substituir a vírgula por ponto decimal.
    return column.str.replace('.', '', regex=False).str.replace(',', '.', regex=False)

In [15]:
def clean_percentage_format(column: pd.Series) -> pd.Series:
    """
    Limpa a formatação de porcentagens em uma coluna do DataFrame.
    
    Parameters:
        column (pandas.Series): A coluna a ser limpa.
    """
    
    # Remover o símbolo de porcentagem e substituir a vírgula por ponto decimal.
    return column.str.replace('%', '', regex=False).str.replace(',', '.', regex=False)

In [16]:
def change_dtype(data_frame: pd.DataFrame,
                 column: str,
                 new_dtype: str) -> pd.DataFrame:
    """
    Altera o tipo de dados de uma coluna do DataFrame.
    
    Parameters:
        data_frame (pandas.DataFrame): O DataFrame contendo os dados.
        column (str): O nome da coluna a ser alterada.
        new_dtype (type): O novo tipo de dados da coluna.
    
    Returns:
        pandas.DataFrame: O DataFrame com a coluna alterada.
    """

    # Mapeamento de tipos especiais
    special_types = {
        'float': lambda col: pd.to_numeric(clean_number_format(col),
                                           errors='coerce'),
        'int': lambda col: col.astype('int'),
        'datetime': lambda col: pd.to_datetime(col,
                                               errors='coerce'),
        'category': lambda col: col.astype('category'),
        'bool': lambda col: col.astype(bool),
        'str': lambda col: col.astype(str)
    }

    try:
        if new_dtype in special_types and new_dtype == 'float':
            if data_frame[column].dtype == 'object' and data_frame[column].str.contains('%').any():
                data_frame[column] = special_types[new_dtype](clean_percentage_format(data_frame[column]))
            else:
                data_frame[column] = special_types[new_dtype](data_frame[column])
        elif new_dtype in special_types:
            data_frame[column] = special_types[new_dtype](data_frame[column])
        else:
            data_frame[column] = data_frame[column].astype(new_dtype)
    except Exception as error:
        print(f"Erro ao converter a coluna '{column}' para {new_dtype}: {error}")

    return data_frame

In [17]:
# Alterando as colunas com dados categóricos para o tipo 'category'.
df = change_dtype(df, 'TipoConsBancario', 'category')
df = change_dtype(df, 'Segmento', 'category')
df = change_dtype(df, 'TipoConsolidacao', 'category')
df = change_dtype(df, 'TipoControle', 'category')

# Alterando a coluna 'DataBase' para o tipo 'datetime'.
df = change_dtype(df, 'DataBase', 'datetime')

# Alterando as colunas com dados numéricos para os tipos 'float'.
df = change_dtype(df, 'AtivoTotal', 'float')
df = change_dtype(df, 'CarteiraCredito', 'float')
df = change_dtype(df, 'PassivoExigivel', 'float')
df = change_dtype(df, 'Captacoes', 'float')
df = change_dtype(df, 'PatrimonioLiquido', 'float')
df = change_dtype(df, 'LucroLiquido', 'float')
df = change_dtype(df, 'PatrimonioReferencia', 'float')

# Alterando as colunas com dados percentuais para o tipo 'float'.
df = change_dtype(df, 'IndiceBasileia', 'float')
df = change_dtype(df, 'IndiceImobilizacao', 'float')

# Alterando as colunas com dados numéricos para os tipos 'int'.
df = change_dtype(df, 'NumAgencias', 'int')
df = change_dtype(df, 'NumPostosAtendimento', 'int')

In [18]:
# Para facilitar compreensão da dimensionalidade dos dados,
# vamos converter os valores monetários para milhões.

# Função lambda para converter os valores para milhões.
change_to_millions = lambda column: column / 1_000

# Aplicando a função lambda para as colunas de valores monetários.
df['AtivoTotal'] = df['AtivoTotal'].apply(change_to_millions)
df['CarteiraCredito'] = df['CarteiraCredito'].apply(change_to_millions)
df['PassivoExigivel'] = df['PassivoExigivel'].apply(change_to_millions)
df['PatrimonioLiquido'] = df['PatrimonioLiquido'].apply(change_to_millions)
df['LucroLiquido'] = df['LucroLiquido'].apply(change_to_millions)

In [19]:
# Alterando as informações da coluna Cidade para que todas as palavras comecem com letra maiúscula.
df['Cidade'] = df['Cidade'].str.title()

In [20]:
# Criando uma nova coluna 'Regiao' com base na coluna 'UF'.
# Obtendo a posição da coluna 'Estado'.
posicao_estado = df.columns.get_loc('UF')
df.insert(posicao_estado, 'Regiao', df['UF'].map(estado_para_regiao))
# Alterando o tipo de dados da coluna 'Regiao' para 'category'.
df = change_dtype(df, 'Regiao', 'category')

In [21]:
# Identificando as colunas que possuem valores 'NI (Não Informado)'.
df.eq('NI').sum().sort_values(ascending=False).rename('NI Count')

Instituicao             0
Codigo                  0
TipoConsBancario        0
Segmento                0
TipoConsolidacao        0
TipoControle            0
Cidade                  0
Regiao                  0
UF                      0
DataBase                0
AtivoTotal              0
CarteiraCredito         0
PassivoExigivel         0
Captacoes               0
PatrimonioLiquido       0
LucroLiquido            0
PatrimonioReferencia    0
IndiceBasileia          0
IndiceImobilizacao      0
NumAgencias             0
NumPostosAtendimento    0
Name: NI Count, dtype: int64

In [22]:
# Verificando a existência de valores nulos no DataFrame.
df.isnull().sum().sort_values(ascending=False).rename('Null Count')

IndiceImobilizacao      134
PatrimonioReferencia     39
IndiceBasileia           39
CarteiraCredito          29
Captacoes                29
PatrimonioLiquido        29
PassivoExigivel          29
AtivoTotal               29
LucroLiquido             29
Segmento                  2
TipoConsBancario          0
Codigo                    0
Instituicao               0
TipoConsolidacao          0
DataBase                  0
Regiao                    0
UF                        0
TipoControle              0
Cidade                    0
NumAgencias               0
NumPostosAtendimento      0
Name: Null Count, dtype: int64

In [23]:
# Filtrando apenas as linhas que possuem 'NaN' como valor.
df[df.isna().any(axis=1)]

,Instituicao,Codigo,TipoConsBancario,Segmento,TipoConsolidacao,TipoControle,Cidade,Regiao,UF,DataBase,AtivoTotal,CarteiraCredito,PassivoExigivel,Captacoes,PatrimonioLiquido,LucroLiquido,PatrimonioReferencia,IndiceBasileia,IndiceImobilizacao,NumAgencias,NumPostosAtendimento
9,NU PAGAMENTOS,1000084693,n1,S3,C,2,Sao Paulo,Sudeste,SP,2024-06-01,214500.65,101837.59,200136.12,134684439.00,14364.53,4069.33,16574240.00,1581.00,NaN,8,0
16,CIELO IP,1000084710,n4,S5,C,2,Barueri,Sudeste,SP,2024-06-01,100538.99,888.42,80147.54,3667690.00,20391.46,1038.62,9116259.00,3454.00,NaN,1,0
22,PAGSEGURO,1000084813,b1,S4,C,2,Sao Paulo,Sudeste,SP,2024-06-01,66937.97,2460.70,55598.37,40615403.00,11339.60,774.18,9491445.00,3336.00,NaN,6,0
24,BANCO MASTER,1000080367,b1,S3,C,2,Rio De Janeiro,Sudeste,RJ,2024-06-01,57152.26,27542.99,52550.65,49522848.00,4601.60,433.03,NaN,NaN,NaN,16,5
27,MERCADO PAGO IP,1000084820,n1,S4,C,2,Osasco,Sudeste,SP,2024-06-01,46794.94,10188.45,43786.93,24492888.00,3008.01,349.11,2626620.00,1451.00,NaN,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1352,COOPERATIVA CENTRAL DE CRÉDITO COM INTERAÇÃO SOLIDÁRIA -...,10013534,b3C,S4,I,2,Lauro De Freitas,Nordeste,BA,2024-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
1353,COOPERATIVA DE CRÉDITO RURAL DO AGRESTE ALAGOANO - COOPE...,8482873,b3S,S5,I,2,Igaci,Nordeste,AL,2024-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
1354,COOPERATIVA DE CRÉDITO MÚTUO DOS SERVIDORES MUNICIPAIS D...,5969937,b3S,S5,I,2,Bebedouro,Sudeste,SP,2024-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
1355,COOPERATIVA DE ECONOMIA E CRÉDITO MÚTUO DOS BANCÁRIOS DO...,1301541,b3S,S5,I,2,Uberlandia,Sudeste,MG,2024-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0


In [24]:
# Substituindo os valores 'NaN' por NA (Not Available).
df = df.fillna(pd.NA)

In [25]:
# Verificando como ficou o DataFrame após as transformações.
df

,Instituicao,Codigo,TipoConsBancario,Segmento,TipoConsolidacao,TipoControle,Cidade,Regiao,UF,DataBase,AtivoTotal,CarteiraCredito,PassivoExigivel,Captacoes,PatrimonioLiquido,LucroLiquido,PatrimonioReferencia,IndiceBasileia,IndiceImobilizacao,NumAgencias,NumPostosAtendimento
0,ITAU,1000080099,b1,S1,C,2,Sao Paulo,Sudeste,SP,2024-06-01,2636882.51,946459.81,2443588.60,1884371314.00,193293.90,19963.58,215556905.00,1656.00,2086.00,2,1
1,BB,1000080329,b1,S1,C,1,Brasilia,Centro-oeste,DF,2024-06-01,2334367.94,1024504.85,2161635.72,1845389366.00,172732.22,18173.03,175348301.00,1419.00,1635.00,4,512
2,CAIXA ECONÔMICA FEDERAL,1000080738,b1,S1,C,1,Brasilia,Centro-oeste,DF,2024-06-01,1907044.01,1175742.24,1807735.64,1617770675.00,99308.37,5300.68,129887763.00,1640.00,926.00,3,846
3,BRADESCO,1000080075,b1,S1,C,2,Osasco,Sudeste,SP,2024-06-01,1682417.43,658552.58,1521287.54,1277833567.00,161129.89,8937.21,149119407.00,1523.00,2557.00,2,1
4,SANTANDER,1000080185,b1,S1,C,3,Sao Paulo,Sudeste,SP,2024-06-01,1279020.31,540556.67,1180740.53,900469651.00,98279.77,6356.90,98725721.00,1438.00,2002.00,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1352,COOPERATIVA CENTRAL DE CRÉDITO COM INTERAÇÃO SOLIDÁRIA -...,10013534,b3C,S4,I,2,Lauro De Freitas,Nordeste,BA,2024-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
1353,COOPERATIVA DE CRÉDITO RURAL DO AGRESTE ALAGOANO - COOPE...,8482873,b3S,S5,I,2,Igaci,Nordeste,AL,2024-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
1354,COOPERATIVA DE CRÉDITO MÚTUO DOS SERVIDORES MUNICIPAIS D...,5969937,b3S,S5,I,2,Bebedouro,Sudeste,SP,2024-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
1355,COOPERATIVA DE ECONOMIA E CRÉDITO MÚTUO DOS BANCÁRIOS DO...,1301541,b3S,S5,I,2,Uberlandia,Sudeste,MG,2024-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0


## Análise Exploratória dos Dados

Após a limpeza e transformação dos dados, podemos realizar uma análise exploratória para obter insights sobre o setor financeiro no Brasil. Vamos começar com uma visão geral dos dados, incluindo o número de instituições financeiras, o tipo de consolidado bancário, o segmento, o tipo de consolidação e o tipo de controle.

In [26]:
def group_and_describe_column(data_frame: pd.DataFrame,
                              column: str,
                              descriptions: dict) -> pd.Series:
    """
    Agrupa e descreve os valores de uma coluna do DataFrame,
    aplicando substituições para torná-los mais legíveis.

    Parameters:
        data_frame (pandas.DataFrame): O DataFrame contendo os dados.
        column (str): O nome da coluna a ser agrupada e descrita.
        descriptions (dict): Um dicionário onde as chaves são os
                             valores originais da coluna e os valores
                             são as descrições legíveis.
    """

    return data_frame[column].replace(descriptions).groupby(
        data_frame[column].replace(descriptions),
                                   observed=True).size().sort_values(ascending=False)

In [27]:
group_and_describe_column(df, 'TipoConsBancario', descricoes_tipo_consolidado)

C:\Users\alexc\AppData\Local\Temp\ipykernel_17732\875438165.py:16: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  return data_frame[column].replace(descriptions).groupby(
C:\Users\alexc\AppData\Local\Temp\ipykernel_17732\875438165.py:17: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  data_frame[column].replace(descriptions),


TipoConsBancario
Cooperativa de Crédito Singular                                          760
Instituição Não Bancária no Mercado de Crédito                           218
Instituição Não Bancária no Mercado de Capitais                          132
Banco Comercial ou Conglomerado com Carteira Comercial                    95
Instituição de Pagamento                                                  83
Banco Múltiplo sem Carteira Comercial ou Banco de Câmbio/Investimento     33
Central e Confederação de Cooperativas de Crédito                         32
Banco de Desenvolvimento                                                   4
Name: TipoConsBancario, dtype: int64

A [Resolução nº 4.553/2017](https://normativos.bcb.gov.br/Lists/Normativos/Attachments/50335/Res_4553_v3_L.pdf) do Banco Central do Brasil define os segmentos para fins da aplicação proporcional da regulação prudencial, considerando o porte e a atividade internacional das instituições. Os segmentos são classificados da seguinte forma:
- S1: Bancos múltiplos, bancos comerciais, bancos de investimento, bancos de câmbio e caixas econômicas que (i) tenham porte (Exposição/Produto Interno Bruto) superior a 10%; ou (ii) exerçam atividade internacional relevante (ativos no exterior superiores a US$ 10 bilhões).
- S2: Composto por: (i) bancos múltiplos, bancos comerciais, bancos de investimento, bancos de câmbio e caixas econômicas de porte inferior a 10% e igual ou superior a 1%; e (ii) demais instituições autorizadas a funcionar pelo Banco Central do Brasil de porte igual ou superior a 1% do PIB.
- S3: Instituições de porte inferior a 1% e igual ou superior a 0,1%.
• S4: Instituições de porte inferior a 0,1%.
- S5: Composto por: (i) instituições de porte inferior a 0,1% que utilizem metodologia facultativa simplificada para apuração dos requerimentos mínimos de Patrimônio de Referência (PR), de Nível I e de Capital Principal, exceto bancos múltiplos, bancos comerciais, bancos de investimento, bancos de câmbio e caixas econômicas; e (ii) não sujeitas a apuração de PR.

In [28]:
df.groupby('Segmento').size().sort_values(ascending=False)

C:\Users\alexc\AppData\Local\Temp\ipykernel_17732\3989073230.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby('Segmento').size().sort_values(ascending=False)


Segmento
S5    912
S4    372
S3     56
S2      8
S1      7
dtype: int64

A maioria das instituições financeiras no Brasil pertence ao segmento S5, que inclui bancos de menor porte e outras entidades autorizadas a operar pelo Banco Central do Brasil, com participação inferior a 0,1% do Produto Interno Bruto (PIB). Analisar apenas essa informação poderia levar à conclusão de que o setor financeiro é diversificado, com uma ampla gama de instituições atendendo a diferentes perfis e necessidades de clientes. No entanto, se considerarmos outras variáveis, como o ativo total e o lucro líquido, uma visão mais detalhada sobre a concentração do setor emerge, revelando o verdadeiro grau de dominância das grandes instituições.

In [29]:
group_and_describe_column(df, 'TipoConsolidacao', descricoes_tipo_consolidacao)

C:\Users\alexc\AppData\Local\Temp\ipykernel_17732\875438165.py:16: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  return data_frame[column].replace(descriptions).groupby(
C:\Users\alexc\AppData\Local\Temp\ipykernel_17732\875438165.py:17: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  data_frame[column].replace(descriptions),


TipoConsolidacao
Instituição Independente    1209
Conglomerado                 148
Name: TipoConsolidacao, dtype: int64

In [30]:
group_and_describe_column(df, 'TipoControle', descricoes_tipo_controle)

C:\Users\alexc\AppData\Local\Temp\ipykernel_17732\875438165.py:16: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  return data_frame[column].replace(descriptions).groupby(
C:\Users\alexc\AppData\Local\Temp\ipykernel_17732\875438165.py:17: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  data_frame[column].replace(descriptions),


TipoControle
Privado Nacional        1247
Controle Estrangeiro      81
Público                   29
Name: TipoControle, dtype: int64

In [31]:
df.groupby('Regiao').size().sort_values(ascending=False)

C:\Users\alexc\AppData\Local\Temp\ipykernel_17732\2180569419.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby('Regiao').size().sort_values(ascending=False)


Regiao
Sudeste         779
Sul             364
Centro-oeste     94
Nordeste         81
Norte            39
dtype: int64

In [32]:
df.groupby('UF').size().sort_values(ascending=False)

UF
SP    474
MG    201
RS    138
SC    114
PR    112
RJ     80
GO     39
ES     24
BA     23
MT     23
DF     19
RO     17
MS     13
PA     12
PB     12
CE     12
PE     10
AL      6
MA      6
SE      4
RN      4
PI      4
AM      3
RR      2
AC      2
TO      2
AP      1
dtype: int64

In [33]:
df.groupby(['Regiao', 'UF']).size().sort_values(ascending=False).head(20) 

C:\Users\alexc\AppData\Local\Temp\ipykernel_17732\3376383296.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby(['Regiao', 'UF']).size().sort_values(ascending=False).head(20)


Regiao        UF
Sudeste       SP    474
              MG    201
Sul           RS    138
              SC    114
              PR    112
Sudeste       RJ     80
Centro-oeste  GO     39
Sudeste       ES     24
Nordeste      BA     23
Centro-oeste  MT     23
              DF     19
Norte         RO     17
Centro-oeste  MS     13
Nordeste      CE     12
              PB     12
Norte         PA     12
Nordeste      PE     10
              MA      6
              AL      6
              PI      4
dtype: int64

In [34]:
df.groupby('Cidade').size().sort_values(ascending=False).head(20)

Cidade
Sao Paulo                296
Rio De Janeiro            65
Porto Alegre              50
Belo Horizonte            48
Curitiba                  36
Barueri                   25
Brasilia                  19
Florianopolis             19
Goiania                   17
Vitoria                   13
Fortaleza                 12
Belem                     10
Salvador                  10
Sao Bernardo Do Campo      9
Cuiaba                     9
Joao Pessoa                8
Osasco                     8
Campo Grande               7
Maringa                    7
Uberlandia                 7
dtype: int64

O sistema financeiro nacional é fortemente concentrado na região sudeste do país, com São Paulo liderando em número de instituições financeiras, seguido por Minas Gerais e Rio de Janeiro, sendo que o Rio de Janeiro ocupa a sexta posição. Essa concentração geográfica pode impactar diretamente a concorrência e a diversificação de produtos e serviços financeiros, limitando o acesso a diferentes opções em outras regiões do Brasil.

## Análise Explanatória
Após obtermos uma compreensão inicial dos dados, podemos avançar para a análise. Neste contexto, nosso objetivo será responder às algumas perguntas:
- *Quais são as instituições financeiras com maior patrimônio líquido?*
- *Qual é a correlação entre o ativo total e o lucro líquido das instituições financeiras?*
- *Como o número de agências e postos de atendimento influencia a carteira de crédito das instituições?*
- *Quais instituições têm os maiores passivos e como isso afeta suas operações?*
- *Qual o impacto das captações no resultado do lucro líquido das instituições financeiras?*
- *Existe uma tendência geográfica nas performances financeiras (como lucros, ativos e patrimônio) de instituições em diferentes estados?*

### Quais são as instituições financeiras com maior patrimônio líquido?

In [35]:
def filter_and_sort_df(data_frame: pd.DataFrame,
                       filter: pd.Series | None = None,
                       columns: list | None = None,
                       sorting: str | None = None,
                       top_n: int = 10) -> pd.DataFrame:
    """
    Filtra, seleciona colunas e ordena um DataFrame.

    Parameters:
        data_frame (pandas.DataFrame): O DataFrame contendo os dados.
        filter (pd.Series): Lista de condições para filtrar o DataFrame.
        columns (list): Lista de colunas a serem selecionadas.
        sorting (str): Coluna para ordenar o DataFrame.
        top_n (int): Número de linhas a serem retornadas.

    Returns:
        pandas.DataFrame: O DataFrame filtrado, com as colunas selecionadas,
                          ordenado e com as 'top_n' linhas.
    """

    if filter is not None:
        data_frame_filtered = data_frame[filter]
    else:
        data_frame_filtered = df

    return data_frame_filtered[columns].sort_values(by=sorting, ascending=False).head(top_n).reset_index(drop=True)

In [36]:
filter_and_sort_df(df,
                   columns=['UF', 'Instituicao', 'AtivoTotal'],
                   sorting='AtivoTotal')

,UF,Instituicao,AtivoTotal
0,SP,ITAU,2636882.51
1,DF,BB,2334367.94
2,DF,CAIXA ECONÔMICA FEDERAL,1907044.01
3,SP,BRADESCO,1682417.43
4,SP,SANTANDER,1279020.31
5,RJ,BNDES,778001.36
6,RJ,BTG PACTUAL,582263.55
7,SP,SAFRA,294923.88
8,RJ,XP,229605.27
9,SP,NU PAGAMENTOS,214500.65


In [37]:
# Removendo os Banco de Desenvolvimento (TipoConsolidado = 'b4') da análise.
filter_and_sort_df(df,
                   filter=df['TipoConsBancario'] != 'b4',
                   columns=['UF', 'Instituicao', 'AtivoTotal'],
                   sorting='AtivoTotal')

,UF,Instituicao,AtivoTotal
0,SP,ITAU,2636882.51
1,DF,BB,2334367.94
2,DF,CAIXA ECONÔMICA FEDERAL,1907044.01
3,SP,BRADESCO,1682417.43
4,SP,SANTANDER,1279020.31
5,RJ,BTG PACTUAL,582263.55
6,SP,SAFRA,294923.88
7,RJ,XP,229605.27
8,SP,NU PAGAMENTOS,214500.65
9,SP,CITIBANK,208735.49


In [38]:
# Considerando apenas as instituições individuais do tipo Banco Comercial,
# Banco Múltiplo com Carteira Comercial ou caixas econômicas e Conglomerado
# composto de pelo menos uma instituição do tipo Banco Comercial,
# Banco Múltiplo com Carteira Comercial ou caixas econômicas.
filter_and_sort_df(df,
                   filter=df['TipoConsBancario'].isin(['b1']),
                   columns=['UF', 'Instituicao', 'AtivoTotal'],
                   sorting='AtivoTotal')

,UF,Instituicao,AtivoTotal
0,SP,ITAU,2636882.51
1,DF,BB,2334367.94
2,DF,CAIXA ECONÔMICA FEDERAL,1907044.01
3,SP,BRADESCO,1682417.43
4,SP,SANTANDER,1279020.31
5,RJ,BTG PACTUAL,582263.55
6,SP,SAFRA,294923.88
7,RJ,XP,229605.27
8,SP,CITIBANK,208735.49
9,RS,BCO COOPERATIVO SICREDI,172849.16


In [39]:
# Considerando apenas as instituições individual do tipo Banco Múltiplo
# sem Carteira Comercial ou Banco de Câmbio ou Banco de Investimento e
# Conglomerado composto de pelo menos uma instituição do tipo Banco
# Múltiplo sem Carteira Comercial ou Banco de Investimento, mas sem conter
# instituições do tipo Banco Comercial e Banco Múltiplo com Carteira Comercial.
filter_and_sort_df(df,
                   filter=df['TipoConsBancario'].isin(['b2']),
                   columns=['UF', 'Instituicao', 'AtivoTotal'],
                   sorting='AtivoTotal')

,UF,Instituicao,AtivoTotal
0,SP,VOLKSWAGEN,46126.60
1,SP,GOLDMAN SACHS,37242.83
2,PR,BANCO CNH INDUSTRIAL CAPITAL S.A,25722.12
3,SP,MERCEDES-BENZ,19965.69
4,SP,BCO CSF S.A.,19363.06
5,RS,BCO DE LAGE LANDEN BRASIL S.A.,16754.44
6,PR,VOLVO,16186.20
7,SP,SCANIA,14919.85
8,SP,STELLANTIS,14685.43
9,SP,GM,13688.97


In [40]:
# Considerando apenas as instituições tipo cooperativa de crédito singular.
filter_and_sort_df(df,
                   filter=df['TipoConsBancario'].isin(['b3S']),
                   columns=['Regiao', 'UF', 'Cidade', 'Instituicao', 'AtivoTotal'],
                   sorting='AtivoTotal')

,Regiao,UF,Cidade,Instituicao,AtivoTotal
0,Sudeste,SP,Bebedouro,COOPERATIVA DE CREDITO CREDICITRUS,15483.29
1,Sul,SC,Blumenau,COOPERATIVA DE CREDITO VALE DO ITAJAI - VIACREDI,13522.38
2,Sudeste,SP,Sertaozinho,SICOOB COCRED COOPERATIVA DE CRÉDITO,12206.65
3,Centro-oeste,MT,Lucas Do Rio Verde,"COOPERATIVA DE CRÉDITO, POUPANÇA E INVESTIMENTO OURO VER...",11845.52
4,Sul,PR,Maringa,"COOPERATIVA DE CRÉDITO, POUPANÇA E INVESTIMENTO DEXIS - ...",11280.02
5,Sul,PR,Palotina,"COOPERATIVA DE CRÉDITO, POUPANÇA E INVESTIMENTO VALE DO ...",9589.15
6,Sudeste,ES,Linhares,COOPERATIVA DE CRÉDITO CONEXÃO - SICOOB CONEXÃO,9464.19
7,Sul,PR,Medianeira,"COOPERATIVA DE CRÉDITO, POUPANÇA E INVESTIMENTO VANGUARD...",9168.10
8,Sul,RS,Nova Petropolis,"COOPERATIVA DE CRÉDITO, POUPANÇA E INVESTIMENTO SICREDI ...",9049.75
9,Sul,RS,Carlos Barbosa,COOPERATIVA DE CRÉDITO SICREDI SERRANA RS/ES,8913.10


As cooperativas singulares com os maiores volumes de ativo total estão predominantemente localizadas na região Sul do Brasil, seguidas pela região Sudeste. Essa concentração reflete a forte tradição e presença do cooperativismo de crédito nessas áreas, demonstrando a importância dessas instituições no oferecimento de serviços bancários à população local.

In [41]:
# Considerando apenas as instituições Central e Confederação de Cooperativas de Crédito.
filter_and_sort_df(df,
                   filter=df['TipoConsBancario'].isin(['b3C']),
                   columns=['Regiao', 'UF', 'Cidade', 'Instituicao', 'AtivoTotal'],
                   sorting='AtivoTotal',
                   top_n=20)

,Regiao,UF,Cidade,Instituicao,AtivoTotal
0,Sudeste,MG,Belo Horizonte,COOPERATIVA CENTRAL DE CRÉDITO DE MINAS GERAIS LTDA. - S...,22272.45
1,Sul,SC,Florianopolis,COOPERATIVA CENTRAL DE CRÉDITO DE SANTA CATARINA E RIO G...,18008.75
2,Sudeste,MG,Belo Horizonte,CENTRAL DAS COOPERATIVAS DE ECONOMIA E CREDITO DO ESTADO...,17048.67
3,Sudeste,SP,Ribeirao Preto,COOPERATIVA CENTRAL DE CRÉDITO DO ESTADO DE SÃO PAULO - ...,13647.49
4,Sul,PR,Francisco Beltrao,COOPERATIVA CENTRAL DE CRÉDITO COM INTERAÇÃO SOLIDÁRIA -...,11855.08
5,Sudeste,ES,Vitoria,COOPERATIVA CENTRAL DE CREDITO DO ESPIRITO SANTO - SICOO...,10968.01
6,Sul,SC,Blumenau,COOPERATIVA CENTRAL DE CRÉDITO - AILOS,10420.12
7,Sul,RS,Porto Alegre,"COOPERATIVA CENTRAL DE CRÉDITO, POUPANÇA E INVESTIMENTO ...",9883.92
8,Sul,PR,Curitiba,"COOPERATIVA CENTRAL DE CRÉDITO, POUPANÇA E INVESTIMENTO ...",6833.22
9,Centro-oeste,GO,Goiania,CENTRAL SICOOB UNI DE COOPERATIVAS DE CRÉDITO,6663.53


As duas maiores centrais em volume de ativo total estão localizadas na cidade de Belo Horizonte, o que demonstra a importância dessa cidade para o setor de cooperativas de crédito. Em contraste, embora São Paulo possua a maior cooperativa em volume de ativos, não há nenhuma cooperativa central situada na cidade de São Paulo entre as vinte primeiras centrais.

In [42]:
# Considerando apenas os bancos de desenvolvimento.
filter_and_sort_df(df,
                   filter=df['TipoConsBancario'].isin(['b4']),
                   columns=['UF', 'Instituicao', 'AtivoTotal'],
                   sorting='AtivoTotal')

,UF,Instituicao,AtivoTotal
0,RJ,BNDES,778001.36
1,RS,BD REGIONAL DO EXTREMO SUL,23202.57
2,MG,BCO DES. DE MG S.A.,9453.71
3,ES,BCO DES. DO ES S.A.,1682.47


In [43]:
# Considerando apenas as instituições não bancárias atuantes no mercado de crédito.
filter_and_sort_df(df,
                   filter=df['TipoConsBancario'].isin(['n1']),
                   columns=['UF', 'Instituicao', 'AtivoTotal'],
                   sorting='AtivoTotal')

,UF,Instituicao,AtivoTotal
0,SP,NU PAGAMENTOS,214500.65
1,SP,MERCADO PAGO IP,46794.94
2,SP,STONE IP,43277.50
3,SP,PORTO SEGURO,18915.28
4,SP,WILL IP,12482.92
5,DF,APE POUPEX,10436.80
6,SP,NEON PAGAMENTOS IP,8703.25
7,SP,PEFISA S.A.,5964.80
8,SP,MIDWAY S.A. - SCFI,5959.83
9,RS,REALIZE CFI,5819.04


In [44]:
# Considerando apenas as instituições não bancárias atuantes no mercado de capitais.
filter_and_sort_df(df,
                   filter=df['TipoConsBancario'].isin(['n2']),
                   columns=['UF', 'Instituicao', 'AtivoTotal'],
                   sorting='AtivoTotal')

,UF,Instituicao,AtivoTotal
0,SP,TERRA INVESTIMENTOS DTVM,27256.70
1,SP,S3 CACEIS BRASIL DTVM,1599.20
2,SP,WISE BRASIL CC,1274.46
3,SP,SINGULARE CTVM,1077.27
4,SP,MIRAE ASSET (BRASIL) CCTVM,605.92
5,SP,GUIDE INVESTIMENTOS,476.90
6,RJ,OLIVEIRA TRUST DTVM S.A.,356.99
7,RJ,VIC DTVM S/A,296.55
8,RJ,BGC LIQUIDEZ DTVM LTDA,284.67
9,SP,TULLETT PREBON BRASIL S.A. CTVM,268.13


In [45]:
# Considerando apenas as instituições de pagamento.
filter_and_sort_df(df,
                   filter=df['TipoConsBancario'].isin(['n4']),
                   columns=['UF', 'Instituicao', 'AtivoTotal'],
                   sorting='AtivoTotal')

,UF,Instituicao,AtivoTotal
0,SP,CIELO IP,100538.99
1,SP,CLOUDWALK IP,11327.05
2,SP,ALELO IP,8163.18
3,SP,PAYPAL DO BRASIL IP,5812.43
4,SP,SHPP BRASIL IP,3061.85
5,SP,HUB IP,2599.43
6,SP,DOCK IP,1896.36
7,SP,ADYEN DO BRASIL IP,1790.20
8,SP,EDENRED SOLUCOES E IP AHA,1675.59
9,SP,IUGU IP,1586.03


Como era de se esperar, a grande maioria das instituições de crédito, capitais e de pagamento está concentrada na capital financeira do país, São Paulo. Essa concentração pode ser explicada pela maior presença de empresas, startups e fintechs na região, que aproveitam a infraestrutura financeira já consolidada para lançar novos produtos e serviços no mercado de forma mais eficiente.

A cidade de São Paulo oferece um ambiente propício para o desenvolvimento do setor financeiro devido à sua ampla rede de investidores, acesso facilitado a capital e mão de obra altamente qualificada. Além disso, a existência de hubs de inovação e incubadoras estimula o surgimento de novas empresas focadas em tecnologia financeira.

In [46]:
df['TipoControle'].isin(['3']).value_counts()

TipoControle
False    1357
Name: count, dtype: int64

In [47]:
# Considerando apenas as instituições financeiras com controle estrangeiro.
filter_and_sort_df(df,
                   filter=df['TipoControle'].isin([3]),
                   columns=['UF', 'Instituicao', 'AtivoTotal'],
                   sorting='AtivoTotal')

,UF,Instituicao,AtivoTotal
0,SP,SANTANDER,1279020.31
1,SP,CITIBANK,208735.49
2,SP,JP MORGAN CHASE,140372.57
3,SP,BNP PARIBAS,91682.98
4,SP,ABC-BRASIL,67603.05
5,SP,BCO RABOBANK INTL BRASIL S.A.,51142.97
6,SP,VOLKSWAGEN,46126.60
7,SP,BCO MUFG BRASIL S.A.,42658.77
8,SP,BOFA MERRILL LYNCH,41690.08
9,SP,GOLDMAN SACHS,37242.83


In [48]:
# Considerando apenas as instituições financeiras com controle privado nacional.
filter_and_sort_df(df,
                   filter=df['TipoControle'].isin([2]),
                   columns=['UF', 'Instituicao', 'AtivoTotal'],
                   sorting='AtivoTotal')

,UF,Instituicao,AtivoTotal
0,SP,ITAU,2636882.51
1,SP,BRADESCO,1682417.43
2,RJ,BTG PACTUAL,582263.55
3,SP,SAFRA,294923.88
4,RJ,XP,229605.27
5,SP,NU PAGAMENTOS,214500.65
6,RS,BCO COOPERATIVO SICREDI,172849.16
7,DF,BANCOOB,146026.20
8,SP,VOTORANTIM,134098.73
9,SP,CIELO IP,100538.99


In [49]:
# Considerando apenas as instituições financeiras com controle público,
# excluindo os bancos de desenvolvimento.
filter_and_sort_df(df,
                   filter=df['TipoControle'].isin([1]) & (
                       df['TipoConsBancario'].isin(['b4']) == False),
                   columns=['UF', 'Instituicao', 'AtivoTotal'],
                   sorting='AtivoTotal')

,UF,Instituicao,AtivoTotal
0,DF,BB,2334367.94
1,DF,CAIXA ECONÔMICA FEDERAL,1907044.01
2,RS,BANRISUL,136280.73
3,CE,BCO DO NORDESTE DO BRASIL S.A.,68248.58
4,PA,BCO DA AMAZONIA S.A.,51049.09
5,ES,BANESTES,38436.73
6,PA,BCO DO EST. DO PA S.A.,18253.22
7,SE,BCO DO EST. DE SE S.A.,10777.03
8,SP,AF DESENVOLVE SP S.A.,4389.80
9,PR,AF PARANÁ,4337.02


Há uma concentração significativa de ativos nos maiores bancos, localizados principalmente nos estados de São Paulo e no Distrito Federal. Instituições privadas, como o Itaú e o Bradesco, sediadas em São Paulo, possuem ativos totais que se equiparam ou até superam os dos grandes bancos públicos, como o Banco do Brasil e a Caixa Econômica Federal, ambos com sede no Distrito Federal. Em contraste, bancos regionais ou estaduais, como o Banrisul (RS) e o Banco do Nordeste do Brasil (CE), apresentam ativos totais substancialmente menores. Esse cenário revela que o setor bancário brasileiro é altamente concentrado, com poucas instituições de grande porte dominando a maior parte dos ativos financeiros, enquanto os bancos regionais têm uma participação significativamente menor no total de ativos do sistema financeiro.

### Qual é a correlação entre o ativo total e o lucro líquido das instituições financeiras?

In [50]:
filter_and_sort_df(df,
                   columns=['Instituicao', 'AtivoTotal'],
                   sorting='AtivoTotal')

,Instituicao,AtivoTotal
0,ITAU,2636882.51
1,BB,2334367.94
2,CAIXA ECONÔMICA FEDERAL,1907044.01
3,BRADESCO,1682417.43
4,SANTANDER,1279020.31
5,BNDES,778001.36
6,BTG PACTUAL,582263.55
7,SAFRA,294923.88
8,XP,229605.27
9,NU PAGAMENTOS,214500.65


In [51]:
filter_and_sort_df(df,
                   columns=['Instituicao', 'LucroLiquido'],
                   sorting='LucroLiquido')

,Instituicao,LucroLiquido
0,ITAU,19963.58
1,BB,18173.03
2,BNDES,13335.00
3,BRADESCO,8937.21
4,SANTANDER,6356.90
5,BTG PACTUAL,5597.06
6,CAIXA ECONÔMICA FEDERAL,5300.68
7,NU PAGAMENTOS,4069.33
8,SAFRA,1381.27
9,CIELO IP,1038.62


In [52]:
# Calculando a rentabilidade dos ativos.
df['RentabilidadeAtivos'] = (df['LucroLiquido'] / df['AtivoTotal']) * 100

In [53]:
filter_and_sort_df(df,
                   columns=['Instituicao', 'RentabilidadeAtivos'],
                   sorting='RentabilidadeAtivos')

,Instituicao,RentabilidadeAtivos
0,COOPERATIVA DE CRÉDITO E INVESTIMENTO COM INTERAÇÃO SOLI...,56.44
1,EAGLE SCD,39.05
2,PENTAGONO S/A DTVM,37.89
3,BIGCASH SCD,28.23
4,WE PAY OUT IP,27.94
5,PROVER PROMOCAO DE VENDAS IP,26.23
6,RED SCD,24.59
7,GDC PART SERV FIDUC DTVM LTDA,24.31
8,SUL AMERICA INVEST DTVM S.A.,24.04
9,INICIADOR IP,23.98


In [54]:
df['RentabilidadeAtivos'].describe()

count   1328.00
mean      -0.23
std       14.88
min     -287.76
25%        0.14
50%        0.95
75%        2.05
max       56.44
Name: RentabilidadeAtivos, dtype: float64

In [55]:
filter_and_sort_df(df,
                   columns=['Instituicao', 'AtivoTotal', 'LucroLiquido', 'RentabilidadeAtivos'],
                   sorting='RentabilidadeAtivos',
                   top_n=20)

,Instituicao,AtivoTotal,LucroLiquido,RentabilidadeAtivos
0,COOPERATIVA DE CRÉDITO E INVESTIMENTO COM INTERAÇÃO SOLI...,0.16,0.09,56.44
1,EAGLE SCD,65.36,25.53,39.05
2,PENTAGONO S/A DTVM,7.17,2.72,37.89
3,BIGCASH SCD,20.38,5.75,28.23
4,WE PAY OUT IP,51.24,14.31,27.94
5,PROVER PROMOCAO DE VENDAS IP,181.60,47.63,26.23
6,RED SCD,7.11,1.75,24.59
7,GDC PART SERV FIDUC DTVM LTDA,3.49,0.85,24.31
8,SUL AMERICA INVEST DTVM S.A.,251.27,60.41,24.04
9,INICIADOR IP,1.64,0.39,23.98


É curioso observar que a maior rentabilidade dos ativos não está necessariamente com as maiores instituições financeiras do país, mesmo que essas instituições possuam grandes volumes de ativos. Por exemplo, a cooperativa de crédito e investimento COOPERATIVA DE CRÉDITO E INVESTIMENTO COM INTERAÇÃO SOLIDÁRIA, apesar de ter um ativo total de apenas 0,16 milhões, apresenta uma impressionante rentabilidade sobre ativos de 56,44%. Em contraste, a SUL AMERICA INVEST DTVM S.A., que possui um ativo total significativamente maior de 251,27 milhões, tem uma rentabilidade de 24,04%, menos da metade da cooperativa. Isso ilustra que a eficiência com que uma instituição usa seus ativos pode variar drasticamente, independentemente de seu tamanho ou volume de ativos, com algumas instituições menores conseguindo maximizar sua rentabilidade de maneira muito eficaz.

In [56]:
# Rentabilidade dos ativos para as instituições dos segmentos S1, S2 e S3.
filter_and_sort_df(df,
                   filter=(df['Segmento'].isin(['S1', 'S2', 'S3'])) & (~df.isnull().any(axis=1)),
                   columns=['Instituicao', 'AtivoTotal', 'LucroLiquido', 'RentabilidadeAtivos'],
                   sorting='RentabilidadeAtivos',
                   top_n=30)

,Instituicao,AtivoTotal,LucroLiquido,RentabilidadeAtivos
0,GM,13688.97,262.23,1.92
1,AGIBANK,24124.10,439.76,1.82
2,BNDES,778001.36,13335.00,1.71
3,COOPERATIVA DE CREDITO VALE DO ITAJAI - VIACREDI,13522.38,227.51,1.68
4,MERCANTIL DO BRASIL,22264.31,347.19,1.56
5,BCO DO NORDESTE DO BRASIL S.A.,68248.58,1022.80,1.50
6,COOPERATIVA DE CREDITO CREDICITRUS,15483.29,213.78,1.38
7,PORTO SEGURO,18915.28,249.19,1.32
8,BCO CSF S.A.,19363.06,239.76,1.24
9,BCO TOYOTA DO BRASIL S.A.,12006.40,139.79,1.16


In [57]:
# Rentabilidade dos ativos para as cooperativas de crédito singular.
filter_and_sort_df(df,
                   filter=df['TipoConsBancario'].isin(['b3S']),
                   columns=['Instituicao', 'AtivoTotal', 'LucroLiquido', 'RentabilidadeAtivos'],
                   sorting='RentabilidadeAtivos',
                   top_n=20)

,Instituicao,AtivoTotal,LucroLiquido,RentabilidadeAtivos
0,COOPERATIVA DE CRÉDITO E INVESTIMENTO COM INTERAÇÃO SOLI...,0.16,0.09,56.44
1,COOPERATIVA DE ECONOMIA E CRÉDITO MÚTUO DOS EMPREGADOS D...,3.04,0.34,11.06
2,COOPERATIVA DE CRÉDITO MÚTUO DOS EMPREGADOS DO MAGAZINE ...,59.62,5.47,9.17
3,GPATIVA - COOPERATIVA DE ECONOMIA E CRÉDITO MÚTUO DOS EM...,157.81,14.12,8.95
4,COOPERATIVA DE ECONOMIA E CRÉDITO MÚTUO DOS EMPREGADOS D...,32.73,2.91,8.90
5,COOPERATIVA DE ECONOMIA E CRÉDITO MÚTUO DOS FUNCIONÁRIOS...,3.48,0.28,7.90
6,COOPERATIVA DE CRÉDITO MÚTUO DOS EMPREGADOS DA SKF E COL...,11.36,0.86,7.57
7,COOPERATIVA DE ECONOMIA E CRÉDITO MÚTUO DOS FUNCIONÁRIOS...,147.01,10.82,7.36
8,COOPERATIVA DE ECONOMIA E CREDITO MUTUO DOS FUNCIONARIOS...,2.62,0.19,7.28
9,COOPERATIVA DE ECONOMIA E CRÉDITO MÚTUO DOS EMPREGADOS D...,104.82,7.62,7.27


In [58]:
# Rentabilidade dos ativos para as centrais e confederações de cooperativas de crédito.
filter_and_sort_df(df,
                   filter=df['TipoConsBancario'].isin(['b3C']),
                   columns=['UF', 'Cidade', 'Instituicao', 'AtivoTotal', 'LucroLiquido', 'RentabilidadeAtivos'],
                   sorting='RentabilidadeAtivos',
                   top_n=20)

,UF,Cidade,Instituicao,AtivoTotal,LucroLiquido,RentabilidadeAtivos
0,SP,Sao Paulo,CONFEDERAÇÃO NACIONAL DAS COOPERATIVAS CENTRAIS UNICRED ...,1204.23,16.75,1.39
1,SP,Sao Paulo,CENTRAL DAS COOPERATIVAS DE CRÉDITO DO ESTADO DE SÃO PAU...,1945.72,17.77,0.91
2,GO,Goiania,"COOPERATIVA CENTRAL DE CRÉDITO DE GOIÁS, DISTRITO FEDERA...",5299.68,37.86,0.71
3,RO,Ji-Parana,CREDISIS - CENTRAL DE COOPERATIVAS DE CRÉDITO LTDA.,1812.91,7.86,0.43
4,BA,Salvador,COOPERATIVA CENTRAL DE CRÉDITO DA BAHIA LTDA - SICOOB CE...,1488.97,6.29,0.42
5,PR,Maringa,CENTRAL DAS COOPERATIVAS DE CREDITO UNICOOB - SICOOB CEN...,6062.78,23.11,0.38
6,PB,Joao Pessoa,"COOPERATIVA CENTRAL DE CRÉDITO, POUPANÇA E INVESTIMENTO ...",4861.45,17.43,0.36
7,GO,Goiania,CENTRAL SICOOB UNI DE COOPERATIVAS DE CRÉDITO,6663.53,22.95,0.34
8,RJ,Rio De Janeiro,COOPERATIVA CENTRAL DE ECONOMIA E CRÉDITO SICOOB UNIMAIS...,2725.57,7.98,0.29
9,RO,Porto Velho,COOPERATIVA CENTRAL DE CRÉDITO DO NORTE DO BRASIL - SICO...,4085.26,11.84,0.29


In [59]:
# As 30 instituições com menor rentabilidade dos ativos.
df[
    ['Instituicao', 'AtivoTotal', 'LucroLiquido', 'RentabilidadeAtivos']
   ].sort_values(by='RentabilidadeAtivos').head(30).reset_index(drop=True)

,Instituicao,AtivoTotal,LucroLiquido,RentabilidadeAtivos
0,NITRO SCD,0.24,-0.68,-287.76
1,N26 SCD,12.05,-28.02,-232.51
2,INIPAY IP,0.17,-0.39,-226.47
3,NVIO BRASIL IP,83.82,-118.92,-141.88
4,MAGNUM SCD,2.48,-2.14,-86.06
5,HBI SCD,1.72,-1.47,-85.37
6,TRANSFERO IP,1.96,-1.65,-84.27
7,CONEXION CC LTDA.,0.36,-0.26,-72.33
8,NOVINVEST CVM LTDA,4.63,-3.14,-67.72
9,HC - DTVM LTDA,0.44,-0.23,-52.52


In [60]:
# AS vinte instituições com pior rentabilidade dos ativos.
filter_and_sort_df(df,
                   filter=(df['RentabilidadeAtivos'] < 0),
                   columns=['Instituicao', 'AtivoTotal', 'LucroLiquido', 'RentabilidadeAtivos'],
                   sorting='RentabilidadeAtivos',
                   top_n=20)

,Instituicao,AtivoTotal,LucroLiquido,RentabilidadeAtivos
0,COOPERATIVA DE CRÉDITO DO NORTE E NOROESTE DE MINAS GERA...,1590.82,-0.05,-0.00
1,GENIAL,4301.52,-0.20,-0.00
2,COOPERATIVA DE ECONOMIA E CRÉDITO MÚTUO DOS EMPREGADOS D...,481.23,-0.04,-0.01
3,SOCIETE GENERALE,30537.16,-4.97,-0.02
4,"COOPERATIVA DE CRÉDITO, POUPANÇA E INVESTIMENTO DOS POLI...",106.17,-0.02,-0.02
5,EWALLY IP,74.93,-0.02,-0.02
6,"COOPERATIVA DE CRÉDITO, POUPANÇA E INVESTIMENTO NORTE MA...",5265.61,-1.32,-0.03
7,COOPERATIVA DE ECONOMIA E CRÉDITO MÚTUO DOS COLABORADORE...,53.83,-0.01,-0.03
8,COOPERATIVA CENTRAL DE CRÉDITO UNICRED GERAÇÃO LTDA. - U...,3760.86,-1.43,-0.04
9,COOPERATIVA DE CRÉDITO UNICRED EVOLUÇÃO LTDA. - UNICRED ...,1521.24,-0.81,-0.05


In [61]:
# Média da rentabilidade de ativos por região do país.
df.groupby('Regiao')['RentabilidadeAtivos'].mean().sort_values(ascending=False)

C:\Users\alexc\AppData\Local\Temp\ipykernel_17732\531332786.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby('Regiao')['RentabilidadeAtivos'].mean().sort_values(ascending=False)


Regiao
Centro-oeste    1.26
Norte           1.20
Sul             0.84
Nordeste        0.63
Sudeste        -1.06
Name: RentabilidadeAtivos, dtype: float64

In [62]:
# Média da rentabilidade de ativos por tipo de consolidado bancário.
df.groupby('TipoConsBancario')['RentabilidadeAtivos'].mean().reset_index().replace(
    {'TipoConsBancario': descricoes_tipo_consolidado}
    ).sort_values(by='RentabilidadeAtivos',ascending=False).reset_index(drop=True)

C:\Users\alexc\AppData\Local\Temp\ipykernel_17732\2537120706.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby('TipoConsBancario')['RentabilidadeAtivos'].mean().reset_index().replace(
C:\Users\alexc\AppData\Local\Temp\ipykernel_17732\2537120706.py:2: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  df.groupby('TipoConsBancario')['RentabilidadeAtivos'].mean().reset_index().replace(


,TipoConsBancario,RentabilidadeAtivos
0,Cooperativa de Crédito Singular,1.35
1,Banco de Desenvolvimento,1.19
2,Banco Múltiplo sem Carteira Comercial ou Banco de Câmbio...,0.78
3,Banco Comercial ou Conglomerado com Carteira Comercial,0.41
4,Central e Confederação de Cooperativas de Crédito,0.24
5,Instituição Não Bancária no Mercado de Capitais,-1.65
6,Instituição Não Bancária no Mercado de Crédito,-3.80
7,Instituição de Pagamento,-5.05


In [63]:
# Média da rentabilidade de ativos por tipo de controle.
df.groupby('TipoControle')['RentabilidadeAtivos'].mean().reset_index().replace(
    {'TipoControle': descricoes_tipo_controle}
    ).sort_values(by='RentabilidadeAtivos',ascending=False).reset_index(drop=True)

C:\Users\alexc\AppData\Local\Temp\ipykernel_17732\3468095964.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby('TipoControle')['RentabilidadeAtivos'].mean().reset_index().replace(
C:\Users\alexc\AppData\Local\Temp\ipykernel_17732\3468095964.py:2: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  df.groupby('TipoControle')['RentabilidadeAtivos'].mean().reset_index().replace(


,TipoControle,RentabilidadeAtivos
0,Controle Estrangeiro,0.71
1,Público,0.29
2,Privado Nacional,-0.30


In [64]:
# Média da rentabilidade de ativos por segmento.
df.groupby('Segmento')['RentabilidadeAtivos'].mean().reset_index().sort_values(
    by='RentabilidadeAtivos', ascending=False).reset_index(drop=True)

C:\Users\alexc\AppData\Local\Temp\ipykernel_17732\3539791361.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby('Segmento')['RentabilidadeAtivos'].mean().reset_index().sort_values(


,Segmento,RentabilidadeAtivos
0,S2,0.64
1,S1,0.63
2,S3,0.58
3,S4,-0.18
4,S5,-0.31


A análise da correlação entre o ativo total e o lucro líquido das instituições financeiras revelou uma tendência positiva, indicando que, em geral, instituições com maiores ativos tendem a apresentar lucros líquidos mais elevados. No entanto, essa relação não é necessariamente linear para todas as instituições, já que fatores como eficiência operacional, gestão de custos e estratégias de captação de recursos podem influenciar significativamente a rentabilidade. Esse insight sugere que, para a alta gestão, além de expandir o volume de ativos, é crucial focar em aprimorar a eficiência operacional e otimizar a alocação de recursos para maximizar o retorno sobre os ativos.